## circular_dilation_raw_cells
The purpose of this notebook is to figure out a good way to turn cell centers detected in raw space into more circular looking objects. Since we don't always have isotropic resolution, it should suffice to have cylindrical cells (for now), i.e the z dilation will simply be with a rectangular kernel. 

The tradeoff in doing this is between how circular something needs to appear and the number of pixels you need to do that. We don't want to use too many pixels because we will lose the ability to distinguish between neighboring cells and also will block out the other layers. On the other hand, we need to use enough pixels so that the cells actually look good enough for publication. A plus sign (technically the smallest non trivial pixelized circle) is not good enough, for example. 

In [17]:
import numpy as np
import os
import csv
import json
from cloudvolume import CloudVolume
import matplotlib.pyplot as plt
import neuroglancer
%matplotlib inline

In [2]:
animal_id = 21
pth=os.path.join('/jukebox/wang/Jess/lightsheet_output',
        '201904_ymaze_cfos','processed',f'an{animal_id}','clearmap_cluster_output',
        'cells.npy')

In [3]:
#first, make a map of cells
converted_points = np.load(pth)

In [15]:
for point in converted_points[0:10]:
    print(point)

[ 570 1516   51]
[ 571 1491   49]
[ 572 1509   51]
[ 577 1388   48]
[ 581 1520   49]
[ 582 1338   49]
[ 582 1392   46]
[ 583 1474   45]
[ 583 1533   51]
[ 584 1434   46]


In [9]:
len(converted_points)

243653

In [16]:
header = "Coordinate 1,Coordinate 2,Ellipsoid Dimensions,Tags,Description,Segment IDs,Parent ID,Type,ID\n"
output_filename = '/home/ahoag/ngdemo/data/cell_annotations_test.csv'
with open(output_filename,'w') as outfile:
    outfile.write(header)
    for point in converted_points[::10]:
        x,y,z=point
        outline=f'''"({x},{y},{z})",,,,,,,Point,\n'''
        outfile.write(outline)
print(f"Wrote {output_filename} ")

Wrote /home/ahoag/ngdemo/data/cell_annotations_test.csv 


In [10]:
# neuroglancer.set_static_content_source(url='https://nglancer.pni.princeton.edu')
neuroglancer.set_static_content_source(url='http://localhost:8080')
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.layers['raw data'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1337'
    )
    s.layers['raw atlas'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338'
    )
print(viewer)

http://127.0.0.1:44272/v/25b96372724aae069488ff1272a4f46ea4fd0c27/


In [11]:
counter=0
with viewer.txn() as s:
    s.layers['annotation']=neuroglancer.AnnotationLayer()
    annotations=s.layers['annotation'].annotations
    
    for (x,y,z) in converted_points[::10]:
    #     print(x,y,z)
        pt=neuroglancer.PointAnnotation(point=[x, y, z],id=f'point{counter}')
        annotations.append(pt)
        counter+=1

## Mask out cells that are not in one of several brain regions
Jess is working with amygdala, thalamus, anterior cingulate. Let's figure out what the numerical ID is for these regions and 

In [13]:
raw_atlas_vol = CloudVolume('file:///home/ahoag/ngdemo/demo_bucket/201904_ymaze_cfos/rawatlas_an21')

In [14]:
raw_atlas_data = np.transpose(raw_atlas_vol[:][...,0],(2,1,0))

Downloading: 100%|██████████| 6183/6183 [01:09<00:00, 110.71it/s]

In [20]:
# Figure out the sub-volume corresponding to the amygdala. To do this, need to figure out all progeny ids 
# Now write the function to get all progeny of an input nodename
def get_progeny(dic,input_nodename,progeny_list):
    """ 
    dic: the dictionary representing the graph
    
    """
    name = dic.get('name')
    children = dic.get('children')
    if name == input_nodename:
        for child in children: # child is a dict
            child_name = child.get('name')
            progeny_list.append(child_name)
            get_progeny(child,input_nodename=child_name,progeny_list=progeny_list)
        return
    
    for child in children:
        child_name = child.get('name')
        get_progeny(child,input_nodename=input_nodename,progeny_list=progeny_list)
    return 

In [18]:
ontology_file = '/home/ahoag/ngdemo/graphviz/allen_ontology_16bit.json'
with open(ontology_file) as json_file:
    data = json.load(json_file)

In [22]:
progeny_list = []
get_progeny(data,input_nodename='Thalamus',progeny_list=progeny_list)
progeny_list

['Thalamus, sensory-motor cortex related',
 'Ventral group of the dorsal thalamus',
 'Ventral anterior-lateral complex of the thalamus',
 'Ventral medial nucleus of the thalamus',
 'Ventral posterior complex of the thalamus',
 'Ventral posterolateral nucleus of the thalamus',
 'Ventral posterolateral nucleus of the thalamus, parvicellular part',
 'Ventral posteromedial nucleus of the thalamus',
 'Ventral posteromedial nucleus of the thalamus, parvicellular part',
 'Posterior triangular thalamic nucleus',
 'Subparafascicular nucleus',
 'Subparafascicular nucleus, magnocellular part',
 'Subparafascicular nucleus, parvicellular part',
 'Subparafascicular area',
 'Peripeduncular nucleus',
 'Geniculate group, dorsal thalamus',
 'Medial geniculate complex',
 'Medial geniculate complex, dorsal part',
 'Medial geniculate complex, ventral part',
 'Medial geniculate complex, medial part',
 'Dorsal part of the lateral geniculate complex',
 'Dorsal part of the lateral geniculate complex, shell',
 

In [25]:
# Now need a dictionary that is the lookup between name and atlas ID
pma_csv_file = '/home/ahoag/ngdemo/graphviz/Princeton_mouse_id_table.csv'
with open(pma_csv_file,'r') as infile:
    reader = csv.reader(infile)
    next(reader)
    pma_id_dict = {row[1]:row[3] for row in reader}

In [26]:
pma_id_dict

{'Tuberomammillary nucleus, ventral part': '1',
 'Primary somatosensory area, mouth, layer 6b': '2',
 'secondary fissure': '3',
 'Inferior colliculus': '4',
 'internal capsule': '6',
 'Principal sensory nucleus of the trigeminal': '7',
 'Basic cell groups and regions': '8',
 'Primary somatosensory area, trunk, layer 6a': '9',
 'Superior colliculus, motor related, intermediate gray layer': '10',
 'posterolateral fissure': '11',
 'Interfascicular nucleus raphe': '12',
 'internal medullary lamina of the thalamus': '14',
 'Parataenial nucleus': '15',
 'Layer 6b, isocortex': '16',
 'Superior colliculus, motor related, intermediate white layer': '17',
 'nodular fissure': '18',
 'Induseum griseum': '19',
 'Entorhinal area, lateral part, layer 2': '20',
 'lateral olfactory tract, general': '21',
 'Posterior parietal association areas': '22',
 'Anterior amygdalar area': '23',
 'simplex fissure': '25',
 'Superior colliculus, motor related, deep gray layer': '26',
 'Intergeniculate leaflet of the

In [58]:
thalamus_id_list = [int(pma_id_dict[prog]) for prog in progeny_list]
thalamus_id_list    

[864,
 637,
 629,
 685,
 709,
 718,
 725,
 733,
 741,
 563807424,
 406,
 414,
 422,
 609,
 1044,
 1008,
 475,
 1072,
 1079,
 1088,
 170,
 496345664,
 496345668,
 496345672,
 856,
 138,
 218,
 1020,
 1029,
 325,
 560581551,
 560581555,
 239,
 255,
 127,
 1096,
 1104,
 64,
 1120,
 1113,
 155,
 444,
 59,
 362,
 617,
 626,
 636,
 366,
 1077,
 571,
 149,
 15,
 181,
 560581559,
 51,
 189,
 599,
 907,
 575,
 930,
 560581563,
 262,
 1014,
 27,
 563807439,
 178,
 300,
 316,
 321,
 958,
 483,
 186,
 953]

In [59]:
# Oddly there is no parent amygdala region in the ontology, so Jess said it is fine to just find all 
# regions with "amygdala" in their name and mask using that
amygdala_id_list = [int(pma_id_dict[name]) for name in pma_id_dict.keys()  if 'amygdal' in name]
amygdala_id_list

[23,
 131,
 192,
 200,
 208,
 216,
 224,
 232,
 240,
 248,
 256,
 278,
 295,
 303,
 311,
 319,
 327,
 334,
 376,
 383,
 400,
 403,
 408,
 411,
 416,
 418,
 424,
 426,
 435,
 451,
 472,
 480,
 487,
 536,
 544,
 551,
 559,
 584,
 592,
 631,
 639,
 647,
 655,
 663,
 780,
 788,
 884,
 1105,
 589508447]

Turns out it is really slow to make a giant OR mask in the volume. Should be faster to do atlas ID lookup when going through the cell coordinates

In [40]:
# Make a giant or mask 
thalamus_mask = np.isin(raw_atlas_data,thalamus_id_list)

KeyboardInterrupt: 

In [42]:
%%time
1 in raw_atlas_data

CPU times: user 1.17 s, sys: 792 ms, total: 1.96 s
Wall time: 1.96 s


True

In [43]:
raw_atlas_data.shape

(687, 2560, 2160)

In [44]:
raw_atlas_data.dtype

dtype('uint16')

In [45]:
%%time
mask = raw_atlas_data == 1

CPU times: user 860 ms, sys: 720 ms, total: 1.58 s
Wall time: 1.57 s


In [46]:
len(thalamus_id_list)

73

In [28]:
ar1=np.array([1,2,3,4])

In [29]:
np.in1d(ar1,[5,5,4,3])

array([False, False,  True,  True])

In [30]:
ar2=np.zeros([2,2])

In [31]:
ar2

array([[0., 0.],
       [0., 0.]])

In [32]:
np.in1d(ar2,[5,5,4,3])

array([False, False, False, False])

In [34]:
ar2[0][1]=5
ar2[1][0]=4
ar2

array([[0., 5.],
       [4., 0.]])

In [36]:
np.isin(ar2,[5,4,3,2])

array([[False,  True],
       [ True, False]])

In [51]:
%%time 
val=raw_atlas_data[600,250,2000]
if val in thalamus_id_list:
    pass
else:
    pass

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 78.7 µs


In [61]:
%%time
for (x,y,z) in converted_points[::100]:
    atlas_id = raw_atlas_data[z,y,x]
    if atlas_id in thalamus_id_list:
        pass
    else:
        pass

CPU times: user 220 ms, sys: 0 ns, total: 220 ms
Wall time: 217 ms


In [66]:
# neuroglancer.set_static_content_source(url='https://nglancer.pni.princeton.edu')
neuroglancer.set_static_content_source(url='http://localhost:8080')
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
#     s.layers['raw data'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1337'
#     )
    s.layers['raw atlas'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338'
    )
print(viewer)

http://127.0.0.1:44272/v/7b1dc34eca1b38969e775126f965a0058aacd9a5/


In [67]:
%%time
counter=0
with viewer.txn() as s:
    s.layers['annotation']=neuroglancer.AnnotationLayer()
    annotations=s.layers['annotation'].annotations
    
    for (x,y,z) in converted_points:
    #     print(x,y,z)
        atlas_id = raw_atlas_data[z,y,x]
        if atlas_id in thalamus_id_list:
            pt=neuroglancer.PointAnnotation(point=[x, y, z],id=f'point{counter}')
            annotations.append(pt)
            counter+=1
        

CPU times: user 19.4 s, sys: 0 ns, total: 19.4 s
Wall time: 19.5 s


So overall it takes about 20 seconds to load in these annotations. 

In [68]:
counter

4476

There are a couple thousand in the thalamus. I expect a similar number in the other regions, maybe slightly fewer based on size. 